In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions
from progress_bar_notebook import ProgressBarNotebook # 自作のJupyter用extension

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
#データセット例をダウンロードできるようになった。これはいつ　のバージョンからかなあ。
train, test = chainer.datasets.get_mnist()

In [3]:
#1.13から入力次元は省略できるようになった。最初に入力されたテストデータで判断するらしい。

class MyChain(Chain):
    def __init__(self):
        super(MyChain, self).__init__(
            c1=L.Convolution2D(None,16, ksize=3, pad=1,), 
            c2=L.Convolution2D(None,64, ksize=3, pad=1),
            c3=L.Convolution2D(None,256, ksize=3, pad=1),
            l1 = L.Linear(4*4*256,512),# 入力をNoneにすると2D to 1D を明示できない
            l2 = L.Linear(None,10),
        )
        
    def __call__(self,x,t):
        return F.softmax_cross_entropy(self.predict(x),t)

    def  predict(self,x):
        h0 = F.reshape(x,(-1,1,28,28)) # 本来はデータセットを二次元にすべきではあるが、ズボラ。
        h1 = F.max_pooling_2d( F.relu(self.c1(h0)),2)
        h2 = F.max_pooling_2d(F.relu(self.c2(h1)),2)
        h3 = F.max_pooling_2d(F.relu(self.c3(h2)),2)
        h4  = self.l1(h3)
        h5  = self.l2(h4)
        return h5

In [4]:
model = MyChain()
optimizer = optimizers.Adam()
optimizer.setup(model)